In [ ]:
from lyricsgenius import Genius

In [ ]:
genius = Genius('A4acQipb4pwRRNdOq06t4A_CYoYBDJR3rCONvPcN84bEwvia3jS8sVq7LnccAfYc')
lafève = genius.search_artist("jul", sort="title",get_full_info=True)
print(lafève.songs)

In [ ]:
n=len(lafève.songs)
data = []
for i in range(n):
    data.append(lafève.songs[i].lyrics)

In [ ]:
cleaned_data = []
for i in data:
    intro_index = i.find("\n")
    cleaned_data.append(i[intro_index+1:])


In [ ]:
lines_of_data = []
for i in range(len(cleaned_data)):
    lines = cleaned_data[i].split("\n")
    filtered_lines = [line for line in lines if not line.startswith('[Refrain') and not line.startswith('[Couplet') and not line.startswith('[Intro') and not line.startswith('[Couplet unique]')]
    paired_lines = [(filtered_lines[i], filtered_lines[i+1]) for i in range(0, len(filtered_lines)-1, 2)]
    lines_of_data.append(paired_lines)

**Now that data is clear, let's fine tune a gpt2 model.**

In [10]:
!pip install transformers


In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer_fr = AutoTokenizer.from_pretrained("antoinelouis/belgpt2")
model_fr = AutoModelForCausalLM.from_pretrained("antoinelouis/belgpt2")

In [2]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments

In [3]:
def load_dataset(file_path, tokenizer, block_size = 128):
    dataset = TextDataset(
    tokenizer = tokenizer,
    file_path = file_path,
    block_size = block_size,
    )
    return dataset

def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=mlm,
    )
    return data_collator

def train(train_file_path,
      output_dir,
      overwrite_output_dir,
      per_device_train_batch_size,
      num_train_epochs,
      save_steps):


    tokenizer = tokenizer_fr
    train_dataset = load_dataset(train_file_path, tokenizer)
    data_collator = load_data_collator(tokenizer)

    tokenizer.save_pretrained(output_dir)

    model = model_fr

    model.save_pretrained(output_dir)

    training_args = TrainingArguments(
          output_dir=output_dir,
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=per_device_train_batch_size,
          num_train_epochs=num_train_epochs,
    )
    trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
    )

    trainer.train()
    trainer.save_model()



In [4]:
pip install accelerate -U

In [5]:
# you need to set parameters
train_file_path = "/content/lines_of_data.txt"
model_name = 'gpt2'
output_dir = '/content/result'
overwrite_output_dir = False
per_device_train_batch_size = 8
num_train_epochs = 5.0
save_steps = 500

In [6]:
train(
    train_file_path=train_file_path,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss
500,3.300100
1000,2.890700
1500,2.623000
2000,2.564900
2500,2.392600
3000,2.368100
3500,2.236800
4000,2.233600
4500,2.131300


In [7]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, GPT2TokenizerFast, GPT2Tokenizer


In [11]:
def generate_text(sequence, max_length):

    model = model_fr
    tokenizer = tokenizer_fr
    ids = tokenizer.encode(f'{sequence}', return_tensors='pt')
    final_outputs = model.generate(
        ids.to('cuda'),
        do_sample=True,
        max_length=max_length,
        pad_token_id=model.config.eos_token_id,
        top_k=50,
        top_p=0.95,
    )
    print(tokenizer.decode(final_outputs[0], skip_special_tokens=True))

In [17]:
sequence = input()
max_len = int(input())
generate_text(sequence, max_len)

Tu l'aimes 
150
Tu l'aimes 
[Pont] Mais où tu vas mon ami
Viens faire un tour, c'est la zone J'reste en indé et au moins on m'dis bravo
L'OVNI et la bande annonce J'ai du stock gros j'vais t'appuyer
J'me régale et j'mets mon perso J'regrette ma team j'ai pas la mort
J'étais seul au quartier avec mes frangins J'suis pas des voyous
Ils aimeraient m'faire passer ma tête en Classe B J'suis pas des gens, j'ai pas d'c'comptes, j'passe pas à l'ess
